<HR>

## Let's get started with Titanic Machine Learning Competition by using <font color="Green">XGBoost</font> and <font color="Blue">Random Forest Classifier</font> and <font color="gold">Deep Learning</font>. Which one is better in this case?

<HR>

### Step1 : Preparation for prediction

In [ ]:
#import libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import plotly.express as px
import xgboost as xgb
import warnings
warnings.simplefilter('ignore')
#read train data and test data
data = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
data

In [ ]:
px.bar(data.query('Survived==1').loc[:,["Sex"]])

In [ ]:
graph2=px.scatter(data.query('Survived==1').loc[:,["Age"]])
graph2

In [ ]:
data[data["Name"].str.contains("Masa")]

### Step2 : Remodel datasets.¶

In [ ]:
#fill missing values drastically 
data = data.fillna(method='bfill')
data = data.fillna(method='ffill')
test = test.fillna(method='bfill')
test = test.fillna(method='ffill')
#make LabeleEncoder into le as an instance
le = LabelEncoder()
#LabelEncode the row named "Sex"
le = le.fit(data['Sex'])
data['Sex'] = le.transform(data['Sex'])
test['Sex'] = le.transform(test['Sex'])
#Change Cabin numbers like C-100 ,A-200 into A , C.  
data['Cabin']=data['Cabin'].str[0:1]
test['Cabin']=test['Cabin'].str[0:1]
#Encode 3 columns into categorical features. 
le = LabelEncoder()
le = le.fit(data['Sex'])
data['Sex'] = le.transform(data['Sex'])
test['Sex'] = le.transform(test['Sex'])
le = le.fit(data['Cabin'])
data['Cabin'] = le.transform(data['Cabin'])
test['Cabin'] = le.transform(test['Cabin'])
le = le.fit(data['Embarked'])
data['Embarked'] = le.transform(data['Embarked'])
test['Embarked'] = le.transform(test['Embarked'])

In [ ]:
x_train= data.iloc[:,[2,4,5,6,7,9,10,11]].astype("int64")
y_train= data.iloc[:,[1]].astype("int64")

In [ ]:
x_train

In [ ]:
y_train

### Step3: Check the correlation in the dataset.

In [ ]:
check_df=pd.concat([x_train,y_train],axis=1)
check_df.corr()

#### Ummm. I decided to use whole columns.¶

In [ ]:
x_test=test.iloc[:,[1,3,4,5,6,8,9,10]]
x_test

In [ ]:
#Normalize data from 0 to 1 by using StandardScaler
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

### Step4 :Let's fit this remodeled dataset into 3 models.

In [ ]:
# XGBoost Regressor
xgbm = xgb.XGBRegressor(max_depth=50)
xgbm.fit(x_train, y_train)
model_score1=xgbm.score(x_train, y_train)
print("The score on traing data of this machine learning model is ",model_score1,"!")

In [ ]:
#Let's fit and predict.
forest = RandomForestClassifier(n_estimators=200, random_state=2, max_depth=300)
forest.fit(x_train, y_train)
model_score2=forest.score(x_train, y_train)
print("The score on traing data of this machine learning model is ",model_score2,"!")

In [ ]:
compare_i=pd.DataFrame(["Random Forest","XGBoost"])
compare_c=pd.DataFrame([model_score2,model_score1])
pd.concat([compare_i,compare_c],axis=1)
             

#### Then,Random Forest Classifier is better?!<HR>

#### How about Deep Learning models? Okay, give it a try. 

<HR>

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras as kr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense, Activation, Flatten,BatchNormalization, Activation,MaxPool2D,Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
np.unique(y_train)

In [ ]:
x_train = x_train.astype('float32')/255
y_train = kr.utils.to_categorical(y_train, 2)

In [ ]:
#Build an ordinary "Deep Learning" model with CNN and maxpooling by using Keras.
model3 = Sequential()
model3.add(Dense(512, input_dim=x_train.shape[1], activation='relu'))
model3.add(Dense(256, activation='relu'))
model3.add(Dense(128, activation='relu'))
model3.add(Dense(64, activation='relu'))
model3.add(Dense(32, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(2, activation='sigmoid'))
model3.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
model3.summary()

In [ ]:
#model fitting
dl_result1=model3.fit(x_train, y_train,batch_size=50, epochs=20)

In [ ]:
#Evaluate the deep learning model by using two metrics, loss and accuracy.
metrics = ['accuracy']
#show the evaluation result by using matoplot.
plt.figure(figsize=(10, 5))
#Use "For Loop".
for i in range(len(metrics)):
    metric = metrics[i]
    #set subplots to show the result
    plt.subplot(1, 2, i+1)
    #Titles of subplots are "loss" and "accuracy"
    plt.title(metric) 
    plt_result1 = dl_result1.history[metric] 
    #plot them all
    plt.plot(plt_result1, label='Deep Learning model') 
    plt.legend() 
plt.show()

### Deep Learning model works a little bit in this case. 
### Anyway, Random Forest Classifier is the best model in 3 models. 

<HR>

### Step5 :Let's make a prediction!?

In [ ]:
y_pred=forest.predict(x_test)

In [ ]:
y_pred2= pd.DataFrame(y_pred)
y_pred2["Survived"]= y_pred2[0]
y_pred2=y_pred2.iloc[:,[1]].astype("int")
y_pred2


### Step6 :Let's compile a submission file.

In [ ]:
#Connect test data and prediction data 
result=pd.concat([test,y_pred2],axis=1)
result=result.iloc[:,[0,11]]
result

In [ ]:
#make an output csv file as submission.csv
result.to_csv('submission_rfc.csv', index=False)
print('submission_file was saved!')

<HR>


#### Thanks for reading my notebook. Feel free to comment :-)

<HR>